# Ceneo Scraper

# Biblioteki

In [ ]:
import requests

## Wysłanie żądania dostępu do zasobu do serwera

In [ ]:
product_id = "114700014"
url = f"https://www.ceneo.pl/{product_id}/opinie-2"
response = requests.get(url)


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
page_dom = BeautifulSoup(response.text, "html.parser")
print(type(page_dom))
opinions = page_dom.select("div.js_product-review")
print(type(opinions))
opinion = opinions.pop(0)
print(type(opinion))
print(opinion)

## Esktrakcja składowych pojedyncztch opinii

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation > em|recommendation|
|gwiazdki|span.user-post__score-count|rating|
|treść|div.user-post__text|content|
|lista zalet|div.review-feature__title--positives ~ div.review-feature__item|pros|
|lista wad|review-feature__title--negatives|cons|
|dla ilu przydatna|span[id^="js_vote-yes"]|useful|
|dla ilu nieprzydatna|span[id^="js_vote-no"]|useless|
|data wystawienia|span.user-post__published > time:nth-child(1)["datetime"]|post_date|
|data zakupu|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|

opinion_id = opinion["data-entry-id"]
author = opinion.select_one("span.user-post__author-name")
